In [39]:
import numpy as np
from scipy.constants import physical_constants as pc
import matplotlib.pyplot as plt


## Puits quantique asymétrique

### d)

Dans un état d'énergie $E$, La partie spatiale de la fonction d'onde obéit à l'équation de schr\"odinger indépendante du temps $\hat{H}\psi(x) =
E\psi(x)$, où l'opérateur hamiltonien $\hat{H}$ est donné par 
\begin{align}
\hat{H} = - {\hbar^2\over2M}\,{d^2\over dx^2} + V(x).
\end{align}

En considérant que le puits de potentiel est infini à $x=0$ et $x=L$, on obtient la fonction d'onde suivante 
\begin{align}
\psi(x) = \sum_{n=1}^\infty \psi_n \sin {\frac{\pi n x}{L}},
\end{align}
où les  $\psi_n$ sont les coefficients de Fourier.

Sachant que pour des entier $m, n$ positifs,
\begin{align}
\int_0^L \sin {\pi m x\over L} \,\sin{\pi n x\over L} dx
  = \begin{cases}
      L/2 & \text{si $m=n$,} \\
      0          & \text{autrement,}
    \end{cases} 
\end{align}

On veut montrer que $\hat{H}\psi = E\psi$ implique

\begin{align}
\sum_{n=1}^\infty \psi_n \int_0^L \sin{\pi m x\over L}
                  \hat{H}\sin{\pi n x\over L} dx
  = \frac{L}{2} E \psi_m.
\end{align}

Procédons.


\begin{align}
\hat{H}\psi(x) = \hat{H}\sum_{n} \psi_n \sin {\frac{\pi n x}{L}} = E\sum_{m} \psi_m \sin {\frac{\pi m x}{L}}
\end{align}

On introduit $\sum_{j}\sin {\frac{\pi j x}{L}}$ de chaque côté, 

\begin{align}
\sum_{n} \sum_{j} \hat{H}\psi_n \sin {\frac{\pi n x}{L}}  \sin {\frac{\pi j x}{L}} = E\sum_{m} \sum_{j}  \psi_m \sin {\frac{\pi m x}{L}} \sin {\frac{\pi j x}{L}}
\end{align}

On intègre ensuite de chaque côté,

\begin{align}
\sum_{n} \sum_{j} \psi_n \int_0^L\sin {\frac{\pi n x}{L}} \hat{H} \sin {\frac{\pi j x}{L}} dx = E\sum_{m} \sum_{j}  \psi_m \int_0^L \sin {\frac{\pi m x}{L}} \sin {\frac{\pi j x}{L}} dx
\end{align}

avec la condition d'orthogonalité et en choisissant $j = m$, 

\begin{align}
\sum_{m} \sum_{n}  \psi_n \int_0^L\sin {\frac{\pi n x}{L}} \hat{H} \sin {\frac{\pi m x}{L}} dx &= \frac{L}{2}E\sum_{m} \psi_m \\
\Rightarrow \sum_{n}  \psi_n \int_0^L\sin {\frac{\pi n x}{L}} \hat{H} \sin {\frac{\pi m x}{L}} dx &= \frac{L}{2}E \psi_m \ \ \ \ \text{C.Q.F.D}
\end{align}


On peut ainsi définir les éléments $H_{mn}$ d'une matrice $\mathbf{H}$ comme
\begin{align}
H_{mn} &= {2\over L} \int_0^L \sin{\pi m x\over L}
          \hat{H}\sin{\pi n x\over L}  dx \nonumber\\
       &= {2\over L} \int_0^L \sin{\pi m x\over L}
          \left[ - {\hbar^2\over2M}\,{d^2\over dx^2} + V(x) \right]
          \sin{\pi n x\over L} dx,
\end{align}

Pour ensuite voir que

\begin{align}
\sum_{n} H_{mn} \psi_n = E \psi_m
\end{align}



### e) 

Avec un potentiel de la forme $V(x) = ax/L$, l'expression de $H_mn$ devient

\begin{align}
H_{mn} &= \frac{2}{L} \int_0^L \sin \left(\frac{m\pi x}{L}\right)\left[\frac{-\hbar^2}{2m} \frac{d^2}{dx^2} + \frac{ax}{L}\right] \sin \left(\frac{n\pi x}{L}\right) dx \\
&= \frac{2}{L} \left\{\underbrace{\int_0^L \sin \left(\frac{m\pi x}{L}\right)\left[\frac{-\hbar^2}{2m} \frac{d^2}{dx^2} \sin \left(\frac{n\pi x}{L}\right)\right]dx}_{(1)} + \underbrace{\int_0^L \sin \left(\frac{m\pi x}{L}\right)\frac{ax}{L} \sin \left(\frac{n\pi x}{L}\right)dx}_{(2)} \right\}\\[0.5cm]
(1) &= \left( \frac{n\pi}{L}\right)^2\frac{\hbar^2}{2m} \int_0^L \sin \left(\frac{m\pi x}{L}\right)\sin \left(\frac{n\pi x}{L}\right)dx\\[0.2cm]
&= 
\begin{cases}
      0 & \text{si}\ m\neq n \\
      \frac{1}{4mL}\left(n\pi\hbar\right)^2 & \text{si}\ m=n
\end{cases}
\\[0.5cm]
(2) &= \frac{a}{L}\int_0^L x\sin \left(\frac{m\pi x}{L}\right) \sin \left(\frac{n\pi x}{L}\right)dx \\[0.2cm]
&= 
\begin{cases}
      0 & \text{si $m\neq n$ et $m, n$ sont pairs} \\
      -\frac{a}{L}\left(\frac{2L}{\pi}\right)^2\frac{mn}{m^2-n^2} & \text{si $m\neq n$ et l'un et l'autre est impair} \\
      \frac{aL}{4}& \text{si $m=n$}
\end{cases}
\end{align}


In [20]:
def get_H_mn(m, n, a=10, L=5e-10):
    hbar = pc['Planck constant over 2 pi'][0]
    m_e = pc['electron mass'][0]
    
    # Résultat du terme de potentiel
    if m != n:

        # Si les deux sont simultanément pairs ou impairs
        if ((m%2 == 0) and (n%2 == 0)) or ((m%2 != 0) and (n%2 != 0)):

            H_mn = 0

        # Si l'un est pair et l'autre est impair, et inversement
        elif ((m%2 == 0) and (n%2 != 0)) or ((m%2 != 0) and (n%2 == 0)):
            H_mn = -(a*4*L/np.pi**2)*(m*n/(m**2 - n**2)**2)
    
    # Si m = n
    else: 
        # Résultat du terme de qte de mvt
        tmp = (n*np.pi*hbar)**2/(4*m_e*L)
        H_mn = a*L/4 + tmp
            
    return 2*(H_mn)/L

H_mn = get_H_mn(2, 1)
print(H_mn)                

-1.8012654869748936


### f) 

In [22]:
def get_H(m, n):
    
    H = np.zeros((m, n))
    
    for i in range(m):
        for j in range(n):
            
            H[i, j] = get_H_mn(i+1, j+1)
            
    return H

H = get_H(10, 10)

#H_dag = H.conj().T

# Vérification Hermitienne
#print(H_dag == H)

eigval, eigvec = np.linalg.eigh(H)
print(eigval)

[ 0.818577    1.74859529  2.67822706  3.60720235  4.53577781  5.46422219
  6.39279765  7.32177294  8.25140471  9.181423  ]


### g)

In [24]:
H_100 = get_H(100, 100)

eigval_100, eigvec_100 = np.linalg.eigh(H_100)

print(eigval[:10])

[ 0.818577    1.74859529  2.67822706  3.60720235  4.53577781  5.46422219
  6.39279765  7.32177294  8.25140471  9.181423  ]


In [50]:
def get_psi_x(mat, n, L=5e-10):
    
    eigvals, eigvecs = np.linalg.eigh(mat)
    
    eigvec = eigvecs[:, n]
    print(eigvec)
    
    x = np.linspace(0, L, 1000)
    psi_x = np.zeros(len(x))
    
    for i in range(1, len(eigvec)+1):
        psi_n = eigvec[i-1]
        psi_x += psi_n*np.sin((np.pi*i*x)/L)
        
    return psi_x, x      
    
psi_1_x, domain = get_psi_x(H, 1)
psi_2_x, _ = get_psi_x(H, 2)
psi_3_x, _ = get_psi_x(H, 3)

plt.plot(domain, psi_1_x)
plt.plot(domain, psi_2_x)
plt.plot(domain, psi_3_x)
plt.show()

[ 0.2257763   0.38544236  0.43196604  0.35249947  0.1699809  -0.06003528
 -0.27090642 -0.39581763 -0.40018321 -0.25865513]
[ 0.32114906  0.42768291  0.24867376 -0.09748098 -0.37912883 -0.41065448
 -0.16937751  0.17788579  0.40323347  0.32757341]
[ 0.39050504  0.33132134 -0.10958669 -0.42287386 -0.24845023  0.21530677
  0.43165078  0.15797189 -0.29571381 -0.37428832]
